In [1]:
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV 
# from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
import os
from functools import reduce
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# read data
path = '/kaggle/input/oil-1th-auc-v2'
test_list = []
train_list = []
for file_name in os.listdir(path):
    f = os.path.join(path,file_name)
    if 'test' in f:
        a1 = pd.read_csv(f)
        a1 = a1.rename(columns={'Y_LABEL_lgb':'Y_LABEL_lgb_'+'_'.join(file_name.split('_')[2:4])})
        test_list.append(a1)
    elif 'train' in f:
        a1 = pd.read_csv(f)
        a1 = a1.rename(columns={'Y_LABEL_lgb':'Y_LABEL_lgb_'+'_'.join(file_name.split('_')[2:4])})
        train_list.append(a1)

In [3]:
# merge dataset
train = reduce(lambda x,y:pd.merge(x,y,on=['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL'],how='left'),train_list)
test = reduce(lambda x,y:pd.merge(x,y,on=['ID','YEAR','COMPONENT_ARBITRARY'],how='left'),test_list)

# drop columns
train = train.drop(columns=['YEAR','COMPONENT_ARBITRARY'])
test = test.drop(columns=['YEAR','COMPONENT_ARBITRARY'])

In [4]:
# 예측값 컬럼 
pred_cols  = sorted(train.columns[train.columns.str.contains('Y_LABEL_')].tolist())

In [5]:
# TARGET=1 blending 값 
a1 = test.copy()
a1[pred_cols] = a1[pred_cols].apply(lambda x: np.where(x>0.2,1,0))
a1['TARGET1'] = a1[pred_cols].apply(lambda x: x.sum(),axis=1)
a1['TARGET1'] = np.where(a1['TARGET1']>0,1,0)
sub_TARGET1 = a1['TARGET1']
print('TARGET1:',a1['TARGET1'].sum())

# TARGET=0 blending 값 
a1 = test.copy()
a1[pred_cols] = a1[pred_cols].apply(lambda x: np.where(x<0.135,1,0))
a1['TARGET0'] = a1[pred_cols].apply(lambda x: x.sum(),axis=1)
a1['TARGET0'] = np.where(a1['TARGET0']>0,1,0)
sub_TARGET0 = a1['TARGET0']
print('TARGET0:',a1['TARGET0'].sum())

TARGET1: 453
TARGET0: 5440


In [6]:
# best모델 호출 
sub = pd.read_csv('/kaggle/input/make-oil-1th/sample_submission_lgb_0.5923.csv')
sub['Y_LABEL'].value_counts()

0    5505
1     536
Name: Y_LABEL, dtype: int64

In [7]:
# TARGET=0인 대상 blending값으로 대체 
sub['Y_LABEL'] = np.where(sub_TARGET0>0,0,sub['Y_LABEL'])
sub['Y_LABEL'].value_counts()

0    5523
1     518
Name: Y_LABEL, dtype: int64

In [8]:
# TARGET=1인 대상 blending값으로 대체 
sub['Y_LABEL'] = np.where(sub_TARGET1>0,1,sub['Y_LABEL'])
sub['Y_LABEL'].value_counts()

0    5519
1     522
Name: Y_LABEL, dtype: int64

In [9]:
# 저장
sub.to_csv('sub_blend_three_model_522.csv',index=False)